In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score,confusion_matrix, accuracy_score
from xgboost import XGBClassifier
from keras.models import Sequential
from keras.layers import Dense

C:\Users\91758\AppData\Local\Continuum\anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\91758\AppData\Local\Continuum\anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
Using TensorFlow backend.


In [2]:
df= pd.read_csv("C:/Users/91758/Downloads/train_LZdllcl.csv")

In [3]:
df.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [4]:
df.isnull().sum()

employee_id                0
department                 0
region                     0
education               2409
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
is_promoted                0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54808 entries, 0 to 54807
Data columns (total 14 columns):
employee_id             54808 non-null int64
department              54808 non-null object
region                  54808 non-null object
education               52399 non-null object
gender                  54808 non-null object
recruitment_channel     54808 non-null object
no_of_trainings         54808 non-null int64
age                     54808 non-null int64
previous_year_rating    50684 non-null float64
length_of_service       54808 non-null int64
KPIs_met >80%           54808 non-null int64
awards_won?             54808 non-null int64
avg_training_score      54808 non-null int64
is_promoted             54808 non-null int64
dtypes: float64(1), int64(8), object(5)
memory usage: 5.9+ MB


In [6]:
df['education'].value_counts()

Bachelor's          36669
Master's & above    14925
Below Secondary       805
Name: education, dtype: int64

In [7]:
df['previous_year_rating'].fillna(0,inplace=True)
df['previous_year_rating'].value_counts()

3.0    18618
5.0    11741
4.0     9877
1.0     6223
2.0     4225
0.0     4124
Name: previous_year_rating, dtype: int64

In [8]:
df['education'].fillna("NAN",inplace=True)

In [9]:
df['education'].value_counts()

Bachelor's          36669
Master's & above    14925
NAN                  2409
Below Secondary       805
Name: education, dtype: int64

In [10]:
le= LabelEncoder()

In [11]:
def lencode(l, df):
    for i in l:
        df[i]= le.fit_transform(df[i])

In [12]:
l=['department','region','education','gender','recruitment_channel']

In [13]:
lencode(l,df)

In [14]:
df.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,7,31,2,0,2,1,35,5.0,8,1,0,49,0
1,65141,4,14,0,1,0,1,30,5.0,4,0,0,60,0
2,7513,7,10,0,1,2,1,34,3.0,7,0,0,50,0
3,2542,7,15,0,1,0,2,39,1.0,10,0,0,50,0
4,48945,8,18,0,1,0,1,45,3.0,2,0,0,73,0


In [15]:
df.gender.value_counts()

1    38496
0    16312
Name: gender, dtype: int64

In [16]:
x= df.drop(['employee_id','is_promoted'], axis=1)

In [17]:
y= df['is_promoted']

In [18]:
xtrain, xtest, ytrain, ytest= train_test_split(x,y, test_size=0.2, random_state=10)

In [19]:
xtrain.shape

(43846, 12)

#### Random Forest Classifier

In [20]:
rf= RandomForestClassifier(n_estimators=50,n_jobs=-1, random_state=42, max_depth=10,max_features='auto',min_samples_leaf=2, min_samples_split=5)
rf.fit(xtrain,ytrain)
pred= rf.predict(xtest)
print(accuracy_score(ytest, pred))
print(f1_score(ytest,pred))

0.9257434774676154
0.26401446654611205


#### XG Boost

In [21]:
xg= XGBClassifier()
xg.fit(xtrain,ytrain)
pred1= rf.predict(xtest)
print(accuracy_score(ytest, pred1))
print(f1_score(ytest,pred1))

0.9257434774676154
0.26401446654611205


#### Neural Networks

In [22]:
model = Sequential()
model.add(Dense(15, input_dim=12, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [23]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
# fit the keras model on the dataset
model.fit(xtrain, ytrain, epochs=180, batch_size=10)

Epoch 1/180
43846/43846 [==============================] - 5s 111us/step - loss: 0.2866 - accuracy: 0.9143
Epoch 2/180
43846/43846 [==============================] - 5s 110us/step - loss: 0.2620 - accuracy: 0.9154
Epoch 3/180
43846/43846 [==============================] - 5s 109us/step - loss: 0.2545 - accuracy: 0.9158
Epoch 4/180
43846/43846 [==============================] - 5s 109us/step - loss: 0.2504 - accuracy: 0.9158
Epoch 5/180
43846/43846 [==============================] - 5s 107us/step - loss: 0.2475 - accuracy: 0.9167
Epoch 6/180
43846/43846 [==============================] - 5s 108us/step - loss: 0.2453 - accuracy: 0.9167
Epoch 7/180
43846/43846 [==============================] - 5s 108us/step - loss: 0.2439 - accuracy: 0.9176
Epoch 8/180
43846/43846 [==============================] - 5s 107us/step - loss: 0.2426 - accuracy: 0.9177
Epoch 9/180
43846/43846 [==============================] - 5s 108us/step - loss: 0.2405 - accuracy: 0.9181
Epoch 10/180
43846/43846 [===========

43846/43846 [==============================] - 5s 111us/step - loss: 0.1887 - accuracy: 0.9334
Epoch 77/180
43846/43846 [==============================] - 5s 110us/step - loss: 0.1881 - accuracy: 0.9332
Epoch 78/180
43846/43846 [==============================] - 5s 107us/step - loss: 0.1871 - accuracy: 0.9338
Epoch 79/180
43846/43846 [==============================] - 5s 106us/step - loss: 0.1879 - accuracy: 0.9337
Epoch 80/180
43846/43846 [==============================] - 5s 106us/step - loss: 0.1874 - accuracy: 0.9336
Epoch 81/180
43846/43846 [==============================] - 5s 106us/step - loss: 0.1874 - accuracy: 0.9337
Epoch 82/180
43846/43846 [==============================] - 5s 106us/step - loss: 0.1871 - accuracy: 0.9332
Epoch 83/180
43846/43846 [==============================] - 5s 106us/step - loss: 0.1864 - accuracy: 0.9338
Epoch 84/180
43846/43846 [==============================] - 5s 106us/step - loss: 0.1867 - accuracy: 0.9338
Epoch 85/180
43846/43846 [===============

43846/43846 [==============================] - 5s 106us/step - loss: 0.1825 - accuracy: 0.9361
Epoch 152/180
43846/43846 [==============================] - 5s 105us/step - loss: 0.1815 - accuracy: 0.9358
Epoch 153/180
43846/43846 [==============================] - 5s 105us/step - loss: 0.1817 - accuracy: 0.9359
Epoch 154/180
43846/43846 [==============================] - 5s 106us/step - loss: 0.1813 - accuracy: 0.9360
Epoch 155/180
43846/43846 [==============================] - 5s 105us/step - loss: 0.1830 - accuracy: 0.9356
Epoch 156/180
43846/43846 [==============================] - 5s 109us/step - loss: 0.1812 - accuracy: 0.9357
Epoch 157/180
43846/43846 [==============================] - 5s 109us/step - loss: 0.1807 - accuracy: 0.9360
Epoch 158/180
43846/43846 [==============================] - 5s 107us/step - loss: 0.1816 - accuracy: 0.9359
Epoch 159/180
43846/43846 [==============================] - 5s 108us/step - loss: 0.1811 - accuracy: 0.9358
Epoch 160/180
43846/43846 [======

In [25]:
# evaluate the keras model
pred3, accuracy = model.evaluate(xtest, ytest)
print('Accuracy: %.2f' % (accuracy*100))

10962/10962 [==============================] - 0s 19us/step
Accuracy: 93.21


#### Test Data 

In [26]:
df_test= pd.read_csv("C:/Users/91758/Downloads/test_2umaH9m.csv")

In [27]:
df_test['previous_year_rating'].fillna(0,inplace=True)

In [29]:
df_test['education'].fillna("NAN",inplace=True)

In [30]:
lencode(l,df_test)

In [31]:
df_test.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,8724,8,18,0,1,2,1,24,0.0,1,1,0,77
1,74430,2,28,0,0,0,1,31,3.0,5,0,0,51
2,72255,7,4,0,1,0,1,31,1.0,4,0,0,47
3,38562,5,11,0,0,0,3,31,2.0,9,0,0,65
4,64486,1,21,0,1,2,1,30,4.0,7,0,0,61


In [36]:
x_test= df_test.drop('employee_id', axis=1)

In [37]:
x_test.shape

(23490, 12)

In [39]:
pred4= model.predict(x_test)

In [41]:
pred4

array([[1.6535220e-01],
       [3.4630299e-05],
       [0.0000000e+00],
       ...,
       [6.4760447e-05],
       [1.0811697e-04],
       [3.4052745e-01]], dtype=float32)

In [40]:
p= pd.Series(pred4)

Exception: Data must be 1-dimensional